In [2]:
from scipy.io import loadmat
from scipy.stats import zscore
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.ndimage import gaussian_filter
from sklearn.manifold import Isomap
%matplotlib


def pcacov(C):
    return np.linalg.eig(C)

def PlotProjections(PCScores,c=[]):

    if not len(c): c = range(np.size(PCScores,0))
    
    plt.figure()
    plt.hsv()
    plt.scatter(PCScores[:,0],PCScores[:,1],1,c = c )
    plt.colorbar()

def PCAnimation(PCScores,c=[],step=500):
    if not len(c): c = range(np.size(PCScores,0))
        
    for i in np.arange(1,np.size(PCScores)-step,5):
        plt.clf()
        plt.scatter(PCScores[i:i+step,0],PCScores[i:i+step,1],1,c = c[i:i+step] )
        plt.draw
        plt.pause(0.0000001)

def PlotProjections3D(PCScores,c=0,step=20):
    if c == 0:
        s = np.size(PCScores,0)
    fig = plt.figure()
    ax = fig.add_subplot(111,projection = '3d')
    ax.scatter(PCScores[0:s:step,0],PCScores[0:s:step,1],PCScores[0:s:step,2],s=1,c=range(s)[0:s:step])

def IsoPlot(X,step,pos = [],sig = 5):
    iso = Isomap(n_components=2)
    Y = iso.fit_transform(X[0:-1:step,:])

    if not len(pos): 
        c = range(len(Y))
    else:
        c = pos[0:-1:step]
    Y[:,0] = gaussian_filter(Y[:,0],sig)
    Y[:,1] = gaussian_filter(Y[:,1],sig)
    plt.figure()
    plt.scatter(Y[:,0],Y[:,1],c=c)


Using matplotlib backend: Qt5Agg


In [3]:
data = loadmat('data_bla')
Qrun = data['Qrun']
Qsws = data['Qsws']
Qrem = data['Qrem']
bins = data['run_bins'].squeeze()
AirpuffDist = data['AirpuffDistance']

data = loadmat('data_hpc')
Qrun = data['Qrun']
Qsws = data['Qsws']
Qrem = data['Qrem']

data = loadmat('pos')
pos = data['pos']

In [4]:
AirpuffDist = pd.DataFrame(AirpuffDist,columns = ['t','d'])
AirpuffDist.index = AirpuffDist.t

Qdist = AirpuffDist.reindex(bins,method= 'nearest').d
Qdist.index = np.arange(0,np.size(Qdist,0),1)

pos = pd.DataFrame(pos,columns=['t','x','y'])

Qpos = pos.reindex(bins,method = 'nearest').x
Qpos.index = np.arange(0,np.size(Qpos,0),1)

print(np.shape(Qpos))

(74115,)


In [5]:
C = np.corrcoef(Qrun.T)
eigvalues,vectors = np.linalg.eig(C)

w1 = vectors[:,0]
w2 = vectors[:,1]
w3 = vectors[:,2]

In [6]:
plt.figure()
plt.stem(range(len(w1)),w1,'b')
plt.stem(range(len(w1)),w2,'r')
plt.stem(range(len(w1)),w3,'g')

<StemContainer object of 3 artists>

In [9]:
PCScoreRun = np.array([zscore(Qrun).dot(w1).T,zscore(Qrun).dot(w2),zscore(Qrun).dot(w3)])
PCScoreRun = gaussian_filter(PCScoreRun.T,[50,0])

PCScoreRem = np.array([zscore(Qrem).dot(vectors[:,0]).T,zscore(Qrem).dot(vectors[:,1])])
PCScoreRem = gaussian_filter(PCScoreRem.T,[50,0])

PCScoreSws = np.array([zscore(Qsws).dot(w1).T,zscore(Qsws).dot(w2),zscore(Qsws).dot(w3)])
PCScoreSws = gaussian_filter(PCScoreSws.T,[50,0])

#PlotProjections3D(PCScoreSws,step = 10)

In [7]:
IsoPlot(Qrun,10,sig = 13)

In [10]:
PlotProjections(PCScoreRun,Qpos)
PlotProjections(PCScoreRem)
PlotProjections(PCScoreSws)

In [10]:
PCAnimation(PCScoreRun)

In [64]:
Qpos

0        124.1066
1        124.1066
2        124.1066
3        124.1066
4        124.1066
           ...   
74110    244.7646
74111    244.7646
74112    244.7646
74113    244.7646
74114    244.7646
Name: x, Length: 74115, dtype: float64

TypeError: _append_dispatcher() missing 1 required positional argument: 'values'